In [1]:
import json
import pandas as pd
import numpy as np
import os
import time
import requests

Update:
1. Epos election happens at the last block of every epoch. However the rpc epos-median number is calculated as if the election happens immediately. We can see the epos-median change in every block, no need to wait until new epoch changes.

2. the median number in the rpc is calculated among all the current validators, not only elected ones. 

In [2]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [3]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [4]:
def getLastBlockOfCurrentEpoch():
    method = 'hmy_getStakingNetworkInfo'
    params = []
    return get_information(method, params)['result']['epoch-last-block']

In [5]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [0]
    return get_information(method, params)['result']

In [6]:
def getEposMedian():
    method = "hmy_getMedianRawStakeSnapshot"
    params = []
    return float(get_information(method, params)['result']['epos-median-stake'])

In [7]:
def get_median(lst):
    n = len(lst) 
    lst.sort() 
    if n % 2 == 0: 
        median1 = lst[n//2] 
        median2 = lst[n//2 - 1] 
        median = (median1 + median2)/2
    else: 
        median = lst[n//2] 
    return median

In [8]:



def M2_test():
    print("Test-M2: Median is correctly computed for even and odd number of available slots")
    num = int(input("enter the number of blocks you want to test: "))
    
    # reach the last block in current epoch to get the eligible validator address
    block = getBlockNumber()
    last_block = getLastBlockOfCurrentEpoch()
    print("current and last block numbers", block, last_block)
    print("wait until the new epoch begins at block number ...", last_block + 1)
    block = getBlockNumber()
    while block < last_block:
        block = getBlockNumber()

    new_epoch_block = block + 1
    while block < new_epoch_block:
        block = getBlockNumber()
    print("new epoch first block reached", new_epoch_block, "will begin testing")
        
    iterations = 0
    while iterations < num:
        print(str(iterations+1) + "iteration will begin ...")
        print("current block number", block)
        # get the median from rpc call
        median = getEposMedian()
        # calculate the median manually
        validator_infos = getAllValidatorInformation()
        stake = []
        for i in validator_infos:
            address = i['validator']['address']
            total_delegation = i['total-delegation']
            if i['metrics'] == None:
                continue
            by_shard_metrics = i['metrics']['by-shard']
            slots = len(by_shard_metrics)
            delegation = total_delegation / slots
#             print("address", address, "total delegation", total_delegation, "slots", slots, "delegation", delegation)
            for by_shard_metric in by_shard_metrics:
                stake.append(delegation)
        cal_median = float(get_median(stake))
        # compare the calculated median and rpc median
        if cal_median != median:
            print("Test-M2: Fail")
            print("manually calculated median stake: " + str(cal_median))
            print("harmony apr call median stake: " + str(median))
            print("")
        else:
            print("Test-M2: Success")
            print("cal_median", cal_median, "rpc median", median)
        
        new_block = getBlockNumber()
        while block == new_block:
            new_block = getBlockNumber()
        block = new_block
        iterations += 1    

In [9]:
M2_test()

Test-M2: Median is correctly computed for even and odd number of available slots
enter the number of blocks you want to test: 1
current and last block numbers 5574 5585
wait until the new epoch begins at block number ... 5586
new epoch first block reached 5586 will begin testing
1iteration will begin ...
current block number 5586
Test-M2: Success
cal_median 2.7174975e+23 rpc median 2.7174975e+23
